In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [3]:
import numpy as np
import pandas as pd

# internal modules
import config

In [4]:
df_train = pd.read_csv(config.jn_datapath + 'train.csv')
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
# storing variable datatypes in a dict.
dict_col_datatypes = {}
for index, value in df_train.dtypes.items():
    if isinstance(value, np.dtypes.Int64DType):
        dict_col_datatypes[index] = 'integer DType'
    elif isinstance(value, np.dtypes.ObjectDType):
        dict_col_datatypes[index] = 'object DType'
    elif isinstance(value, np.dtypes.Float64DType):
        dict_col_datatypes[index] = 'float DType'
    else:
        raise "We haven't accounted for at least one datatype"
        
assert df_train.shape[1] == len(dict_col_datatypes), "We haven't accounted for at least one datatype"

In [6]:
# columns with at least 1 missing value and their datatypes:
df_cols_missing_val_cnt = df_train.isna().sum(axis=0)
df_cols_missing_val_cnt = df_cols_missing_val_cnt.loc[df_cols_missing_val_cnt>0]
df_cols_missing_val_cnt = df_cols_missing_val_cnt.rename_axis('feature_names').reset_index(name='missing_value_count')
df_cols_missing_val_cnt['datatype'] = df_cols_missing_val_cnt['feature_names'].apply(lambda x: 
                                                                   dict_col_datatypes.get(x, 'unknown DType'))
df_cols_missing_val_cnt

,feature_names,missing_value_count,datatype
0,LotFrontage,259,float DType
1,Alley,1369,object DType
2,MasVnrType,872,object DType
3,MasVnrArea,8,float DType
4,BsmtQual,37,object DType
5,BsmtCond,37,object DType
6,BsmtExposure,38,object DType
7,BsmtFinType1,37,object DType
8,BsmtFinType2,38,object DType
9,Electrical,1,object DType


In [ ]:
# Imouting missing data
